In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/app')

In [3]:
from vexa import VexaAPI

In [4]:
voyager_apikey = 'pa-g4x1yHL4Hv6qBjXcrT2Nhm5gG0srlDhVBp6Op7_IJBQ'

In [5]:
vexa = VexaAPI()
meetings = await vexa.get_meetings()

Vexa token: 3ae04e20124d40babc5107e658c666b6
Request URL: http://host.docker.internal:8001/api/v1/calls/all
Request Params: {'token': '3ae04e20124d40babc5107e658c666b6'}


In [6]:
meetings[-1]['id']

'6a3451aa-f18c-4211-8140-fbc77ea7ba05'

In [7]:
df, formatted_output, start_datetime, speakers,transcript = await vexa.get_transcription(meeting_session_id=meetings[-1]['id'])


In [8]:
input_text = df[['formatted_time','speaker', 'content']].to_markdown()
input_text


'|     | formatted_time   | speaker             | content                                                                                                                                                                                                                                                                                                                                                                                                                                                           |\n|----:|:-----------------|:--------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [9]:
from core import BaseCall,system_msg,user_msg,count_tokens


In [10]:
from pydantic import Field

In [11]:
class TopicsMapping(BaseCall):
    formatted_time: str = Field(..., description="time of the input qoute")
    topic: str = Field(..., description="topic of the input qoute")

In [12]:
class TopicsExtraction(BaseCall):
   # topics: list[str] = Field(..., description="List of topics extracted from the input text")
    mapping: list[TopicsMapping] = Field(..., description="Mapping of topics to the input text. Aim for equal chunks of time for each topic switch. ")

In [13]:
r = await TopicsExtraction.call([system_msg(f"Extract topics from the following text: {input_text}"),user_msg(input_text)])
r


TopicsExtraction(mapping=[TopicsMapping(formatted_time='00:00', topic='Introduction and Questions'), TopicsMapping(formatted_time='00:09', topic='Product Development'), TopicsMapping(formatted_time='01:16', topic='Product Features and Structure'), TopicsMapping(formatted_time='02:12', topic='Chat and Information Management'), TopicsMapping(formatted_time='02:34', topic='Meeting Notes and Transcripts'), TopicsMapping(formatted_time='03:50', topic='Knowledge Hub Concept'), TopicsMapping(formatted_time='04:20', topic='Onboarding Process'), TopicsMapping(formatted_time='05:10', topic='User Statistics and Metrics'), TopicsMapping(formatted_time='09:38', topic='Retention and User Engagement'), TopicsMapping(formatted_time='12:00', topic='Monetization Potential'), TopicsMapping(formatted_time='13:07', topic='Product Packaging and Business Model'), TopicsMapping(formatted_time='15:01', topic='Focus on Onboarding and Monetization'), TopicsMapping(formatted_time='19:19', topic='Traffic and User 

In [14]:
r.mapping

[TopicsMapping(formatted_time='00:00', topic='Introduction and Questions'),
 TopicsMapping(formatted_time='00:09', topic='Product Development'),
 TopicsMapping(formatted_time='01:16', topic='Product Features and Structure'),
 TopicsMapping(formatted_time='02:12', topic='Chat and Information Management'),
 TopicsMapping(formatted_time='02:34', topic='Meeting Notes and Transcripts'),
 TopicsMapping(formatted_time='03:50', topic='Knowledge Hub Concept'),
 TopicsMapping(formatted_time='04:20', topic='Onboarding Process'),
 TopicsMapping(formatted_time='05:10', topic='User Statistics and Metrics'),
 TopicsMapping(formatted_time='09:38', topic='Retention and User Engagement'),
 TopicsMapping(formatted_time='12:00', topic='Monetization Potential'),
 TopicsMapping(formatted_time='13:07', topic='Product Packaging and Business Model'),
 TopicsMapping(formatted_time='15:01', topic='Focus on Onboarding and Monetization'),
 TopicsMapping(formatted_time='19:19', topic='Traffic and User Acquisition')

In [15]:
import pandas as pd

In [16]:
# Convert mapping to DataFrame
topics_df = pd.DataFrame([{"formatted_time": m.formatted_time, "topic": m.topic} for m in r.mapping])

# Merge with original df
df = df.merge(topics_df, on='formatted_time', how='left')

In [17]:
df

,index,speaker,speaker_id,content,html_content,html_content_short,keywords,timestamp,formatted_time,time_tuple,initials,chunk_number,topic
0,0,Viktoria Kharlamova,TBD,С чем у вас? Какие вопросы?,С чем у вас возникли вопросы?,,[],2024-12-13T11:30:30.795999Z,00:00,"(0, 0)",VK,0,Introduction and Questions
1,1,Viktoria Kharlamova,TBD,Есть ли нужная помощь?,Есть ли <b>нужная помощь</b>?,нужная помощь,[],2024-12-13T11:30:32.635999Z,00:01,"(0, 1)",VK,0,NaN
2,2,Viktoria Kharlamova,TBD,"Ну,",None,None,[],2024-12-13T11:30:34.075999Z,00:03,"(0, 3)",VK,0,NaN
3,3,Dmitry Grankin,TBD,полностью все пересобралось.,None,None,[],2024-12-13T11:30:40.455999Z,00:09,"(0, 9)",DG,0,Product Development
4,4,Dmitry Grankin,TBD,Но в основном это разработка.,Но в основном это <b>разработка</b>.,разработка,[],2024-12-13T11:30:46.855999Z,00:16,"(0, 16)",DG,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,197,,TBD,"Good. Тогда рада была поболтать. Вот, надеюсь...",None,None,[],2024-12-13T11:58:05.399999Z,27:34,"(27, 34)",NaN,35,Closing Remarks and Future Collaboration
198,198,,TBD,получится заимплементировать классную монетиз...,None,None,[],2024-12-13T11:58:10.471999Z,27:39,"(27, 39)",NaN,35,NaN
199,199,,TBD,"начать растить продукты, потом, если понадоби...",None,None,[],2024-12-13T11:58:14.591999Z,27:43,"(27, 43)",NaN,35,NaN
200,200,,TBD,"Давай, спасибо тебе большое.",None,None,[],2024-12-13T11:58:15.931999Z,27:45,"(27, 45)",NaN,35,NaN


In [18]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [19]:
df = df[['formatted_time','topic','content','speaker']].fillna(method='ffill')


/tmp/ipykernel_455143/2833176756.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df[['formatted_time','topic','content','speaker']].fillna(method='ffill')


In [20]:
df['speaker_shift'] = (df['speaker']+df['topic'] != df['speaker']+df['topic'].shift(1)).cumsum()


In [21]:
df_grouped = df.groupby('speaker_shift').agg({
    'formatted_time': 'first',
    'topic': 'first',
    'speaker': 'first',
    'content': ' '.join
}).reset_index()

In [23]:
df_grouped

,speaker_shift,formatted_time,topic,speaker,content
0,1,00:00,Introduction and Questions,Viktoria Kharlamova,"С чем у вас? Какие вопросы? Есть ли нужная помощь? Ну,"
1,2,00:09,Product Development,Dmitry Grankin,"полностью все пересобралось. Но в основном это разработка. Я тебе покажу сейчас, как продукт выглядит. А я им пользуюсь, кстати. вами в августе по моему созванивались вот тогда я себе скачал и какое-то время попробовала вот но сейчас может тут другой уже ну да в общем здесь с точки зрения расширения Во время встречи у нас, наверное, не сильно что-то изменилось. А во время встречи у нас получается интересный продукт."
2,3,01:16,Product Features and Structure,Dmitry Grankin,"совокупности, которые выглядят таким образом. У нас сюда вываливаются встречи, короткое summary, hit points of interest и у нас упорно чат у нас чат по встрече чат по спикеру можно взять какого-то отдельного спикера и про него все. Прикольно. В общем, все структурнее стало, получается. Да. Вот. Есть глобальный чат. Да, можно..."
3,4,02:12,Chat and Information Management,Dmitry Grankin,"здесь несколько спикеров выбрать и по ним почататься и есть прям глобальный такой чат, с которым можно разговаривать можно глобальные вопросы любые задавать то есть мы пытаемся сделать такой клод и в котором в которой информация в проекты будут попадать автоматически и есть есть такой продукт Есть внутри проекта. Проект — это когда ты туда складываешь информацию, документы, и по ним получаешь чат по этим документам. Мы хотим, чтобы не надо было копипать. Никакие документы никуда, когда ты определяешь контекст двумя кнопками, вот, и тебе это все доступно здесь. Такая концепция."
4,5,02:34,Meeting Notes and Transcripts,Viktoria Kharlamova,"Правильно я понимаю, что в том числе Meeting Notes, они будут подтягиваться туда или пока что нет? Meeting Notes, куда туда будут подтягиваться? Но имеется в виду, смотри, То есть то, что я помню, каким образом выглядел продукт, это были в основном meeting notes. Ну, то есть я ассистент, который записывал у тебя то, о чем разговаривали на встрече. Транскрипты, да, вот, вот. Да, да, да. Транскрипт, который вычищенный и с хайлайтами. Да, и здесь они есть, да, вот наш транскрипт вот здесь. И получается, что помимо этого, то есть этот транскрипт будет подтягиваться сюда и автоматически добавляться, то есть либо можно будет быстро добавить что-то в определенные проекты, я правильно понимаю? Ну, именно проекты. Да, он автоматически уже здесь, наш транскрипт уже здесь, пока мы говорим. Все здесь. Дальше концептуально. Мы этот output или какой-то другой текст добавляем сюда же в проекты, какие-то отчеты, которые тоже... накапливаются и становятся доступны для чата."
5,6,03:50,Knowledge Hub Concept,Dmitry Grankin,"Вот, какие-то внешние источники данных. То есть это такой, по сути, knowledge hub проектовых. где вся информация хранится и может быть структурирована. Здесь какие-то визуалки. Да, этот сайт я посмотрела, то есть ваше главное, как поменялось. Да."
6,7,04:20,Onboarding Process,Dmitry Grankin,"Chrome Extension, если... Про анбординг говорить. Установили там три кнопки и попали в дашборд. Дашборд или звонок."
7,8,05:10,User Statistics and Metrics,Dmitry Grankin,"по тому что у нас с пользователями происходит со статистикой давай прям покажу чтобы не рассказывать что это такое я лучше в таких сумму статистику ты сама делала вывод насколько это смотри пока ищешь быстро задам еще вопрос то есть концептуально если я правильно понимаю, то то, куда вы движетесь, это какой-то умный хаб всех знаний внутри компании, всех проектов. Вот. Ну, соответственно, вы просто будете больше структурировать. Мы движемся от идеи митинг-ноутейкера, базовая потребность, которая закрывает митинг-ноутейкер, людей это том что я записала она у меня там есть я потом как-то на этом увернуться могу вернуться потом они по сути дела практически и не возвращаются вот мы да мы хотим дальше есть какая концепция есть Там есть продукты типа ChatGPT, Clot AI, это продукты, кото

In [22]:
doc_content = input_text

In [23]:
chunks = (df_grouped['speaker']+': '+df_grouped['content']).tolist()

In [24]:
chunk_content = chunks[0]

In [25]:
DOCUMENT_CONTEXT_PROMPT = """
<document>
{doc_content}
</document>
"""

CHUNK_CONTEXT_PROMPT = """
Here is the chunk we want to situate within the whole document
<chunk>
{chunk_content}
</chunk>

Please give a short succinct context to situate this chunk within the overall document for the purposes of improving search retrieval of the chunk.
Answer only with the succinct context and nothing else.
"""

In [26]:
from core import generic_call

In [27]:
messages=[
    system_msg(DOCUMENT_CONTEXT_PROMPT.format(doc_content=doc_content)),
    user_msg(CHUNK_CONTEXT_PROMPT.format(chunk_content=chunks[8]))
]


In [28]:
r = await generic_call(messages)
r

This chunk is part of a conversation between Viktoria Kharlamova and Dmitry Grankin discussing the performance metrics of a product, specifically focusing on user retention, active users, and the challenges related to monetization and banking issues. It highlights the need for increased traffic and user engagement while assessing the current state of the product's market fit.


"This chunk is part of a conversation between Viktoria Kharlamova and Dmitry Grankin discussing the performance metrics of a product, specifically focusing on user retention, active users, and the challenges related to monetization and banking issues. It highlights the need for increased traffic and user engagement while assessing the current state of the product's market fit."

In [29]:
r

"This chunk is part of a conversation between Viktoria Kharlamova and Dmitry Grankin discussing the performance metrics of a product, specifically focusing on user retention, active users, and the challenges related to monetization and banking issues. It highlights the need for increased traffic and user engagement while assessing the current state of the product's market fit."

In [30]:
embedding_model = 'voyage-3'

In [36]:
import voyageai
embedder = voyageai.Client(voyager_apikey)

In [38]:
embs =embedder.embed(texts=[r],model=embedding_model)


In [39]:
embs.embeddings

[[-9.544720523990691e-05,
  -0.05848415195941925,
  0.032721832394599915,
  -0.05763135850429535,
  0.02380470000207424,
  0.03552868589758873,
  0.04359779506921768,
  -0.040484555065631866,
  -0.03456682711839676,
  0.01381328608840704,
  0.016482030972838402,
  -0.016294250264763832,
  -0.01400077622383833,
  0.03735280781984329,
  -0.030846793204545975,
  -0.004263709764927626,
  0.03412386775016785,
  -0.0021854296792298555,
  0.013573437929153442,
  0.04384612292051315,
  -0.011139207519590855,
  -0.0431826077401638,
  0.029953718185424805,
  0.013988138176500797,
  -0.03997860103845596,
  0.012174582108855247,
  -0.003882906399667263,
  -0.003375261789187789,
  -0.0231022909283638,
  0.04868150129914284,
  -0.0032280038576573133,
  0.01817345805466175,
  -0.002009018324315548,
  -0.052579957991838455,
  0.013179806061089039,
  -0.009144174866378307,
  0.04477320611476898,
  -0.007760605309158564,
  -0.018109427765011787,
  -0.05329589545726776,
  -0.015043308958411217,
  -0.0094

In [ ]:
from 